<a href="https://colab.research.google.com/github/santa-dask/aai-540-in1-fin-int-ai-agent/blob/main/Untitled26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!git clone https://github.com/santa-dask/aai-540-in1-fin-int-ai-agent

In [4]:
#%cd /content/aai-540-in1-fin-int-ai-agent/
%pwd
%pip install -U -q  -r setup/requirement.txt --extra-index-url https://pypi.org/simple   --no-warn-script-location

Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.


In [1]:
import sys
import os
from dotenv import load_dotenv


# Explicitly set project_root to the cloned repository path
#project_root = '/content/aai-540-in1-fin-int-ai-agent'
project_root = '.'
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

from utils import config_loader as cl
from ingestion.setup_infra  import setup_data_lake
from ingestion import load_raw_data
from preprocessing import preprocess
from model_training import gemma_training

### Set up default credential

In [2]:
project_id = cl.config_loader.get(cl.PROJECT_ID)
region = cl.config_loader.get(cl.REGION)

print(project_id)
print(region)

setup_data_lake(project_id)
gemma_training.setup_model_training_environment(cl)
#print("Successfully called ")

cfpb-usd-aai-540
us-central1
 GCS Bucket already exists: cfpb-usd-aai-540-cfpb-raw-lake
 GCS Bucket already exists: cfpb-usd-aai-540-cfpb-clean-features
 GCS Bucket already exists: cfpb-usd-aai-540-cfpb-weights-registry
BigQuery Dataset already exists: cfpb-usd-aai-540.cfpb_analysis
Setting up model training environment for google/gemma-3-4b-it 
CUDA driver not found. Falling back to standard attention.


In [11]:
%pwd

'c:\\Users\\Shivam\\aai-540-in1-fin-int-ai-agent'

In [3]:
import pandas as pd
#raw_file_path = f"../{cl.config_loader.get(cl.RAW_FILE_PATH)}"
raw_file_path=f'./data/complaints1.csv'
print(raw_file_path)
complaints_ds = pd.read_csv(raw_file_path)
sample_ds = complaints_ds.head(100)
sample_ds.to_csv('./data/complaints2.csv')

./data/complaints1.csv


In [2]:
PROJECT = cl.config_loader.get(cl.PROJECT_ID)
RAW_BUCKET = f"{PROJECT}-{cl.config_loader.get(cl.RAW_BUCKET)}"
RAW_FILE_PATH =  f"./{cl.config_loader.get(cl.RAW_FILE_PATH)}" # Ensure file exists here

load_raw_data.load_to_raw_bucket(PROJECT, RAW_BUCKET, RAW_FILE_PATH)

⏳ Uploading ./data/complaints1.csv to gs://cfpb-usd-aai-540-cfpb-raw-lake/raw/complaints1.csv...
✅ Success! File available at: gs://cfpb-usd-aai-540-cfpb-raw-lake/raw/complaints1.csv


In [2]:
PROJECT = cl.config_loader.get(cl.PROJECT_ID)
RAW_BUCKET = f"{PROJECT}-{cl.config_loader.get(cl.RAW_BUCKET)}"
DATASET = cl.config_loader.get(cl.BQ_DATASET)

preprocess.preprocess_raw_to_staging (PROJECT, DATASET, RAW_BUCKET)

 GCS RAW File Location: gs://cfpb-usd-aai-540-cfpb-raw-lake/raw/complaints1.csv


✅ Preprocessing complete. Staging table created: cfpb-usd-aai-540.cfpb_analysis.stg_complaints


### Split Dataset

In [4]:
complaints_ds = preprocess.preprocess_data(cl)
preprocess.split_data(cl, complaints_ds,10)

Number of records before removing records with Nan value : 13212301


### Test performance of the vanilla model

In [10]:
sample_file_path=cl.config_loader.get(cl.SAMPLED_FILE_PATH)
sample_dataset = gemma_training.load_and_clean_data(sample_file_path)

In [ ]:
for row in sample_dataset.take(2):
    print(row)

IndexError: Index 4 out of range for dataset of size 2.

In [7]:
sample_tokeniser = gemma_training.get_tokeniser(cl)
sample_model = gemma_training.get_model(cl)
print(f"Device Used: {sample_model.device}")

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

c:\Users\Shivam\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Shivam\.cache\huggingface\hub\models--google--gemma-3-4b-it. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

RuntimeError: Data processing error: CAS service error : ReqwestMiddleware Error: Request failed after 5 retries

In [ ]:
sample_texts = sample_dataset["text"]
ppl_score_vanilla_model = gemma_training.calculate_perplexity(sample_model, sample_tokeniser, sample_texts )

In [ ]:
print(f" Perplexity Score of plain vanilla gemma3b model : {ppl_score_vanilla_model.item():.4f}")

### Domain Adaptive Pretraining using LoRA

In [ ]:
training_file_path=cl.config_loader.get(cl.TRAINING_FILE_PATH)
training_dataset = gemma_training.load_and_clean_data(training_file_path)

In [ ]:
training_tokeniser = gemma_training.get_tokeniser(cl)
training_model = gemma_training.get_model(cl)
print(f"Device Used: {training_model.device}")

In [ ]:
#def tokenize_function(examples):
#    # Financial narratives can be long; H100 handles 1024+ context easily
#    return training_tokeniser(examples["text"], truncation=True, max_length=512)

def tokenize_function(examples):
    return training_tokeniser(
        examples["text"],
        truncation=True,
        max_length=512,
        padding="max_length", # Consistent length helps with mask mapping
        return_token_type_ids=True
    )
tokenized_dataset = training_dataset.map(
        tokenize_function,
        batched=True,
        num_proc=8, # Multiprocess tokenization for speed
        remove_columns=["text"]
)

# Apply the update
tokenized_dataset = training_dataset.map(tokenize_function, batched=True)


In [ ]:
gemma_training.clear_memory()

In [ ]:
gemma_training.get_training( training_model, training_tokeniser, tokenized_dataset)

### Performance testing of DAPT model

In [ ]:
sample_texts = sample_dataset["text"]
ppl_score_vanilla_model = gemma_training.calculate_perplexity(sample_model, sample_tokeniser, sample_texts )
print(f" Perplexity Score of plain vanilla gemma3b model : {ppl_score_vanilla_model.item():.4f}")
